<a href="https://colab.research.google.com/github/mandresblanco/Salud_Chile/blob/main/Salud_chile_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introducción.**
 El síndrome metabólico (SM) es un grupo de cuadros que ponen en riesgo a una persona de desarrollar una enfermedad cardiaca y diabetes tipo 2. Estos cuadros son: (i) Hipertensión arterial; (ii) Glucosa alta en la sangre; (iii) Niveles sanguíneos elevados de triglicéridos; (iv) Bajos niveles sanguíneos de colesterol HDL; y (v) Exceso de grasa alrededor de 
la cintura (circunferencia de cintura). 



**Objetivo.** Caracterizar y determinar factores asociados a SM en Chile, usando datos de la 
Encuesta Nacional de Salud (ENS) 2009-2010. 


Diseño de la ENS. La encuesta consideró una muestra de 5412 personas, usando estratificación 
por regiones y según zona urbana – rural de cada región, el tamaño muestral por región se 
eligió en forma proporcional al número de habitantes de cada región. Al interior de cada región 
se seleccionaron al azar conglomerados correspondientes a las comunas y, dentro de las 
comunas, se eligieron al azar unidades censales (áreas definidas para el Censo de Población del 
INE). Finalmente, al interior de las unidades censales se seleccionaron hogares usando 
muestreo aleatorio simple. Se seleccionó una submuestra al azar de 4800 
individuos de la ENS2009.

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks/Projects'
!ls

/content/drive/My Drive/Colab Notebooks/Projects
'Base ENS2009 Reducida.xlsx'		    kiva_country_profile_variables.csv
 country_profile_variables.csv		    Salud_Chile.csv
 country_vaccinations_by_manufacturer.csv   Salud_Chile.xlsx
 country_vaccinations.csv		    vaccines.csv
 dfColSex.xlsx


In [3]:
df = pd.read_excel('Base ENS2009 Reducida.xlsx')
df

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN
1,2333,71,1,1.0,1.0,113.0,166.0,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0
2,403,61,1,1.0,1.0,114.9,162.0,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0
3,1059,68,2,1.0,2.0,92.0,161.0,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0
4,3310,46,2,1.0,2.0,89.4,150.0,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


IMC : Peso / Talla2

In [4]:
df.columns 
df['talla'] = df['talla']/100
df['IMC'] = df['peso'] / df['talla']**2
df


,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ETARIO : EDAD en rangos: 

1 para ≤20,

2 para 21-40,

3 para 41-60, 

4 para >60 años


In [5]:
# df.reset_index()
# df = df.drop('oms', axis=1)
cond = [
               (df['edad']<=20),
               (df['edad']>20) & (df['edad']<41),
               (df['edad']>40) & (df['edad']<61),
               (df['edad']>60)]
opt = [1,2,3,4]
df['ETARIO'] = np.select(cond,opt)
df
# i=0



,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


SCORE_CV : Suma de factores de riesgo cardiovascular (CV): Edad>60, IMC>30, 
 pas130_pad85=1, Col_Total>200 y hdl40_50=1. 

el score va de 0 a 5

5 cuando cumple todos los factores de riesgo y 0 cuando no cumple ninguno

In [6]:

for i in range(len(df)):
  k=0
  if df.loc[i, 'edad'] > 60:
    k=k+1
  if df.loc[i, 'IMC'] >30.0:
    k=k+1
  if df.loc[i, 'pas130_pad85'] ==1.0:
    k=k+1
  if df.loc[i, 'col_total'] >200.0:
    k=k+1
  if df.loc[i, 'hdl40_50'] ==1.0:
    k=k+1
  # print(k)
  df.loc[i,'SCORE_CV'] = k
df


# cond = [
#                (df['edad']>60) & (df['IMC']>30) & (df['pas130_pad85']==1) & (df['Col_Total']>200) & (df['hdl40_50']==1),
#                (df['edad']>20) & (df['edad']<41),
#                (df['edad']>40) & (df['edad']<61),
#                (df['edad']>60)]
# opt = [1,2,3,4]
# df['SCORE_CV'] = np.select(cond,opt)
# df.head(50)

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO,SCORE_CV
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4,1.0
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4,4.0
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4,4.0
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4,4.0
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4,1.0
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0


CUAL_CV : Todas las posibles combinaciones en que se presentan los 5 factores 
 usados para construir SCORE_CV. 


In [7]:

for i in range(len(df)):
  if df.loc[i, 'edad'] > 60:
    k1 = '/edad '
  if df.loc[i, 'IMC'] >30.0:
    k1 = k1 + '/IMC '
  if df.loc[i, 'pas130_pad85'] ==1.0:
    k1 = k1 + '/pas130 '
  if df.loc[i, 'col_total'] >200.0:
    k1 = k1 + '/col '
  if df.loc[i, 'hdl40_50'] ==1.0:
    k1 = k1 + '/hd140 '
  df.loc[i,'CUAL_CV'] = k1
  # print(k1)
df

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO,SCORE_CV,CUAL_CV
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4,1.0,/edad
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4,4.0,/edad /IMC /pas130 /hd140
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4,4.0,/edad /IMC /pas130 /hd140
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4,4.0,/edad /IMC /pas130 /hd140
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3,3.0,/edad /IMC /pas130 /hd140 /IMC /pas130 /hd140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4,1.0,/edad
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad


In [ ]:
dir_pandas='/content/drive/My Drive/Colab Notebooks/Projects/{}'
dir_pandas
df.to_excel(dir_pandas.format('Salud_Chile.xlsx'))

In [8]:
df['ones']=1
df

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO,SCORE_CV,CUAL_CV,ones
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4,1.0,/edad,1
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4,4.0,/edad /IMC /pas130 /hd140,1
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4,4.0,/edad /IMC /pas130 /hd140,1
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4,4.0,/edad /IMC /pas130 /hd140,1
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3,3.0,/edad /IMC /pas130 /hd140 /IMC /pas130 /hd140,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad,1
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad,1
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4,1.0,/edad,1
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad,1


In [120]:
df_sm = df[['sm', 'ones', 'edad']]
df_sm = df_sm.dropna()
df_sm = df_sm.reset_index()
df_sm = df_sm.drop('index', axis=1)

for i in range(3300):
  if df_sm.loc[i, 'edad'] < 18:
    df_sm = df_sm.drop([i], axis=0) 
    
df_sm = df_sm.reset_index()
df_sm = df_sm.drop('index', axis=1)
df_sm

,sm,ones,edad
0,1.0,1,71
1,1.0,1,61
2,1.0,1,68
3,1.0,1,46
4,0.0,1,23
...,...,...,...
3136,0.0,1,19
3137,0.0,1,18
3138,0.0,1,74
3139,0.0,1,31


In [121]:

df_sm.to_excel(dir_pandas.format('SmMayor.xlsx'))

In [ ]:
dir_pandas='/content/drive/My Drive/Colab Notebooks/Projects/{}'
dir_pandas
df.to_excel(dir_pandas.format('Salud_Chile.xlsx'))

In [ ]:
# max(df['edad'])
# min(df['edad'])
# max(df['glucosa'])
# min(df['glucosa'])
max(sd)
# min(sd)

75.24289575571628

In [9]:

df_col_sex = df[['sexo', 'col_hdl']]
# df_col_sex=df_col_sex.groupby('sexo')['col_hdl']


In [10]:
for i in range(len(df_col_sex['sexo'])):
  if df.loc[i, 'sexo'] == 1:
    df_col_sex.loc[i,'sexo'] = 'Masc'
  if df.loc[i, 'sexo'] == 2:
    df_col_sex.loc[i,'sexo'] = 'Fem'

df_col_sex

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,sexo,col_hdl
0,Fem,NaN
1,Masc,28.0
2,Masc,32.0
3,Fem,45.0
4,Fem,48.0
...,...,...
4795,Fem,NaN
4796,Fem,NaN
4797,Fem,NaN
4798,Masc,NaN


In [11]:
df_col_sex = df_col_sex.dropna()
df_col_sex

,sexo,col_hdl
1,Masc,28.0
2,Masc,32.0
3,Fem,45.0
4,Fem,48.0
5,Fem,62.0
...,...,...
4752,Masc,61.0
4758,Fem,97.0
4769,Masc,33.0
4777,Fem,54.0


In [12]:
df_col_sex = df_col_sex.reset_index()
df_col_sex = df_col_sex.drop('index', axis=1)
df_col_sex

,sexo,col_hdl
0,Masc,28.0
1,Masc,32.0
2,Fem,45.0
3,Fem,48.0
4,Fem,62.0
...,...,...
2474,Masc,61.0
2475,Fem,97.0
2476,Masc,33.0
2477,Fem,54.0


In [62]:
df_col_sex['ones']=1
df_col_sex
# np.percentile(df_col_sex['sexo'], 50)


,sexo,col_hdl,ones
0,Masc,28.0,1
1,Masc,32.0,1
2,Fem,45.0,1
3,Fem,48.0,1
4,Fem,62.0,1
...,...,...,...
2474,Masc,61.0,1
2475,Fem,97.0,1
2476,Masc,33.0,1
2477,Fem,54.0,1


In [61]:
# df_col = df_col_sex.groupby('sexo')['col_hdl', 'ones'].sum()
# df_col
km = np.zeros(1007)
# k1[]
kf = np.zeros(1472)
m=0
km[m] = 28
m= m+1
f=0
for i in range(1,2478+1):
   if df_col_sex.loc[i, 'sexo'] == 'Masc':
     km[m] = df_col_sex.loc[i, 'col_hdl']
     m=m+1
   if df_col_sex.loc[i, 'sexo'] == 'Fem':
     kf[f] = df_col_sex.loc[i, 'col_hdl']
     f=f+1
print(km,kf)

[28. 32. 65. ... 49. 61. 33.] [45. 48. 62. ... 97. 54. 60.]


In [78]:
df_m = pd.DataFrame(km, columns=(['Nivel_col_Masc']))
df_m = df_m.sort_values('Nivel_col_Masc',ascending=True)
df_f = pd.DataFrame(kf, columns=(['Nivel_col_Fem']))
df_f = df_f.sort_values('Nivel_col_Fem',ascending=True)
df_m = df_m.reset_index()
df_m = df_m.drop('index', axis=1)
df_m
df_f = df_f.reset_index()
df_f = df_f.drop('index', axis=1)
print(df_m, df_f)

      Nivel_col_Masc
0               17.0
1               21.0
2               21.0
3               21.0
4               21.0
...              ...
1002            81.0
1003            82.0
1004            82.0
1005            84.0
1006           176.0

[1007 rows x 1 columns]       Nivel_col_Fem
0              20.0
1              23.0
2              23.0
3              25.0
4              25.0
...             ...
1467           97.0
1468          101.0
1469          106.0
1470          106.0
1471          108.0

[1472 rows x 1 columns]


In [95]:
k=[5, 10, 25, 50, 75, 90, 95]
l=0
m= np.zeros(7)
f= np.zeros(7)
for i in k:
  m[l]=np.percentile(df_m, i, axis=0)
  f[l]=np.percentile(df_f, i, axis=0)
  l= l+1
print("Perc", k)
print('Masc',m)
print('Feme',f)

Perc [5, 10, 25, 50, 75, 90, 95]
Masc [28.  31.  36.  42.  51.  59.  65.7]
Feme [32. 35. 41. 48. 57. 66. 74.]


In [79]:
dir_pandas='/content/drive/My Drive/Colab Notebooks/Projects/{}'
dir_pandas
df_m.to_excel(dir_pandas.format('NivelColMasc.xlsx'))
df_f.to_excel(dir_pandas.format('NivelColFem.xlsx'))

Buscando la media de las proporciones

In [152]:
a=np.zeros(1000)
for i in range(1000):
  a[i]=sm.sample(740).mean()
m=0
for i in a:
  m=i+m
print(m/1000)

0.32186756756756796


In [ ]:

df_col_sex.to_excel(dir_pandas.format('dfColSex.xlsx'))